枝セグメントのうち葉隠れ部分を色変えてマスク画像作成

In [ ]:
# CIVE = 0.441 * R - 0.811 * G + 0.385 * B + 18.78745
# cive > 0なら緑
# wood(0, 224, 192), leaves(128, 32, 0)

import os
import cv2
import numpy as np
import shutil

def create_color_mask(image, original_mask):
    b, g, r = cv2.split(image)

    cive = 0.441 * r - 0.811 * g + 0.385 * b + 18.78745
    ExG = 2 * g - r - b

    leaf_condition = ((cive > 0) | (ExG < 0))
    leaf_mask = leaf_condition & (original_mask > 0)
    wood_mask = (~leaf_condition) & (original_mask > 0)

    # 出力画像を3チャンネルに変更
    result_image = np.zeros_like(image)
    
    result_image[leaf_mask] = [0, 32, 128]
    result_image[wood_mask] = [192, 224, 0]

    return result_image

def process_images(line2line_path, gendata_path, output_path):
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path, exist_ok=True)

    for filename in os.listdir(line2line_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            line2line_image_path = os.path.join(line2line_path, filename)
            gendata_image_path = os.path.join(gendata_path, filename)

            line2line_image = cv2.imread(line2line_image_path, 0)
            gendata_image = cv2.imread(gendata_image_path)

            mask = np.uint8(line2line_image > 128)

            if np.sum(mask) > 0:
                colored_mask = create_color_mask(gendata_image, mask)
                output_file = os.path.join(output_path, filename)
                cv2.imwrite(output_file, colored_mask)
    

def main():
    mask_path = './masks/'
    gendata_path = './grape_test/images/'
    output_path = './colored_masks/'

    process_images(mask_path, gendata_path, output_path)

if __name__ == '__main__':
    main()